__GDSC2__:

- nr of rows
- nr of unique cell lines: `809`
- nr of unique drug names: `192`
- nr of unique drug ids: `198`
- dist of ln_ic50
- how many rows have ic50 values, how many haven't
- dist of uniq drugs per cell line
    - [ ] which are the cell lines with the highest number of unique drugs? how much more do they have compared to mean and median?
- count of drugs in total. 
    - [ ] which are the top 10 occuring drugs?

__Final dataset used for training__:
- nr of rows: `137,835`
    - [ ] how can this be higher than all rows in GDSC2 `135,242`?
    - [ ] shouldn't we sparse down the dataset a bit? why isn't it sparsed down
- nr of unique cell lines
- nr of unique drugs
- dist of ln_ic50
- dist of uniq drugs per cell line

__Gene Expression__:
- which dataset and from where? 

--- 

- [ ] Change whole pipeline to ONLY inlcude GDSC2. Currently I included GDSC1 and GDSC2
- [ ] Then run a model for `combined_score_thresh` of 700, 750, 800, 850, 900, 950, 990

In [ ]:
!pwd